In [2]:
from Annotation_helper import *

In [3]:
transactions = load_transactions("../../Data/structured_data\VOC Cochin Slave Transactions 1706-1801 - IISH - Database - 2018 - v1.csv")
permissions = load_permissions("../../Data/structured_data\VOC Cochin Slave Transport Permissions 1770-1795 - IISH - Database - 2018 - v1.csv")

In [4]:
per_to_trans = {
                "BezitterVoornaam" : ("KoperVoornaam", 2),
                "BezitterTussen": ("KoperTussen", 1),
                "BezitterAchternaam" : ("KoperAchternaam", 2),
                "SlaafNaamNieuw" : ("SlaafNaamNieuw", 2),
                "BezitterBeroep" : ("KoperBeroep", 1),
                "BezitterStatus" : ("KoperStatus", 1),
                "SlaafGender" : ("SlaafGender", 1),
                "Datum" : ("Datum", 1),
                "BezitterWerkplek" : ("KoperWerkplek", 1),
                "SlaafKaste" : ("SlaafKaste", 1)
}

In [5]:
get_n_dict_value(per_to_trans).values()

dict_values(['KoperVoornaam', 'KoperTussen', 'KoperAchternaam', 'SlaafNaamNieuw', 'KoperBeroep', 'KoperStatus', 'SlaafGender', 'Datum', 'KoperWerkplek', 'SlaafKaste'])

In [6]:
transactions[transactions["ID"] == "896-321-1"][get_n_dict_value(per_to_trans).values()]

,KoperVoornaam,KoperTussen,KoperAchternaam,SlaafNaamNieuw,KoperBeroep,KoperStatus,SlaafGender,Datum,KoperWerkplek,SlaafKaste
4404,Philip,NaN,Philips,October,NaN,Vrijman,m,1771-08-03,NaN,Namboe


In [7]:
permissions[permissions["ID"] == "897-3-21"][per_to_trans.keys()]

,BezitterVoornaam,BezitterTussen,BezitterAchternaam,SlaafNaamNieuw,BezitterBeroep,BezitterStatus,SlaafGender,Datum,BezitterWerkplek,SlaafKaste
74,Philip,NaN,Philipsz:,October,NaN,Vrijman,m,1770-10-12,NaN,NaN


In [8]:
fuzz.ratio("Phillips", "Phillipz")

88

In [9]:
fuzz.ratio("Phillips", "Phillips")

100

In [14]:
print(transactions.iloc[:,1:-2].head().to_latex())

\begin{tabular}{lllllllllllllllllllllllllllll}
\toprule
{} &       ID &       Datum & VerkoperVoornaam & VerkoperPatroniem & VerkoperTussen & VerkoperAchternaam & VerkoperGender & VerkoperHerkomst & VerkoperBeroep &    VerkoperWerkplek & VerkoperEtniciteit & VerkoperStatus & KoperVoornaam & KoperPatroniem & KoperTussen & KoperAchternaam & KoperGender & KoperHerkomst & KoperBeroep &     KoperWerkplek & KoperEtniciteit & KoperStatus & SlaafNaamNieuw & SlaafNaamOrigineel & SlaafTaalNaam & SlaafKaste & SlaafGender & SlaafLeeftijd \\
\midrule
0 &  554-1-1 &  1753-01-02 &          Michiel &               NaN &            NaN &             Kruijs &              m &              NaN &      Metselaar &              Cochin &                NaN &            VOC &        Gillis &            NaN &         NaN &          Dabijn &           m &           NaN &    Capitain &  Schip Sloterdijk &             NaN &         VOC &       December &                NaN &           NaN &        NaN &          

In [16]:
print(permissions.head().to_latex())

\begin{tabular}{llrrrlllrlllllllllllllll}
\toprule
{} &       ID &  Permissie &  Inventarisnummer &  Scan &  Plaats &       Datum & BezitterVoornaam &  BezitterPatroniem & BezitterTussen & BezitterAchternaam & BezitterGender & BezitterHerkomst & BezitterBeroep & BezitterWerkplek & BezitterEtniciteit & BezitterStatus & SlaafNaamNieuw & SlaafNaamOrigineel & SlaafKaste & SlaafGender & Opmerking &                                           Betaling & Door[Kantlijn] \\
\midrule
0 &  897-1-1 &          1 &               897 &     1 &  Cochin &  1770-08-06 &              NaN &                NaN &            NaN &                NaN &            NaN &              NaN &            NaN &              NaN &                NaN &            NaN &         Tutius &                NaN &        NaN &           m &       NaN &  Mits betalende geregtigheijd aan den pagter en... &            NaN \\
1 &  897-1-2 &          2 &               897 &     1 &  Cochin &  1770-08-06 &              NaN &         

# Translation table

In [13]:
pd.DataFrame(get_n_dict_value(per_to_trans), index=[0])

,BezitterVoornaam,BezitterTussen,BezitterAchternaam,SlaafNaamNieuw,BezitterBeroep,BezitterStatus,SlaafGender,Datum,BezitterWerkplek,SlaafKaste
0,KoperVoornaam,KoperTussen,KoperAchternaam,SlaafNaamNieuw,KoperBeroep,KoperStatus,SlaafGender,Datum,KoperWerkplek,SlaafKaste


In [24]:
res = pd.DataFrame.from_dict(get_n_dict_value(per_to_trans), orient='index').reset_index().rename(columns={"index": "permission dataset", 0: "transaction dataset"})

In [28]:
print(res.to_latex(index=False))

\begin{tabular}{ll}
\toprule
permission dataset & transaction dataset \\
\midrule
  BezitterVoornaam &       KoperVoornaam \\
    BezitterTussen &         KoperTussen \\
BezitterAchternaam &     KoperAchternaam \\
    SlaafNaamNieuw &      SlaafNaamNieuw \\
    BezitterBeroep &         KoperBeroep \\
    BezitterStatus &         KoperStatus \\
       SlaafGender &         SlaafGender \\
             Datum &               Datum \\
  BezitterWerkplek &       KoperWerkplek \\
        SlaafKaste &          SlaafKaste \\
\bottomrule
\end{tabular}

